# Funciones Base

Para manipular los pixeles de una imagen, primero se debe acceder a ellos. Con este fin usamos la librería Image:

In [1]:
from PIL import Image
from random import randint

# Ojo que no funciona con .jpg
im = Image.open("dog_img.png")

print 'Size:', im.size
x = im.size[0]
y = im.size[1]

Size: (960, 650)


Tenemos una imagen de "x" pixeles de ancho e "y" de alto. La información de los colores (R,G,B) de cada pixel se obtiene:

In [2]:
pix = im.load()
test = [randint(0, x), randint(0, y)]
print pix[test[0], test[1]]

(8, 21, 3, 255)


Es decir, pix[test[0], test[1]] muestra los colores [R,G,B] del pixel random recién testeado. Cada color, se representa con un número que va desde el 0 hasta el 255, es decir, por color se tienen 2^8 tonalidades. Esto último, se puede representar con 8-bits por color.

In [3]:
def int_to_bin(int):
    b = bin(int)[2:] # Primero 2 términos siempre son "0b"
    if len(b) < 8:
        dif = 8 - len(b)
        b = '0' * dif + b
    return b

int_test = randint(0, 255)
print 'Test: Nº', int_test, 'Binary:', int_to_bin(int_test)

Test: Nº 200 Binary: 11001000


Ahora, podemos representar los colores de cada pixel usando bits:

In [4]:
def pixel_bin(pix, pix_test):
    bp = [int_to_bin(pixel) for pixel in pix[pix_test[0], pix_test[1]]]
    return bp

test = [randint(0, x), randint(0, y)]
print 'Test: Pixel:', test
print '(RGB) en enteros:', pix[test[0], test[1]]
print '(RGB) en binario:', pixel_bin(pix, test) 

Test: Pixel: [822, 539]
(RGB) en enteros: (4, 16, 2, 255)
(RGB) en binario: ['00000100', '00010000', '00000010', '11111111']


Por otra parte, los caracteres ASCII están representados por un número entero que va desde 0 al 255 (¡qué coincidencia!). Por lo tanto, cada cáracter se pasa a entero y luego a binario.

In [5]:
def char_int_bin(c):
    entero = ord(c) # ord(x) convierte el cáracter c en int.
    binario = int_to_bin(entero)
    return binario

test = "hola"
for c in test:
    print 'Char:', c, 'Int:', ord(c), 'Bin:', char_int_bin(c)

Char: h Int: 104 Bin: 01101000
Char: o Int: 111 Bin: 01101111
Char: l Int: 108 Bin: 01101100
Char: a Int: 97 Bin: 01100001


# Algoritmo (a)

## Términos Modificables

El algoritmo (a) dice: <i>"Cada pixel oculta un carácter, siendo los 3 bits de los 2 primeros colores y los 2 últimos del último color"</i>. Osea, cada cáracter se compone de 8 bits, los cuales, del primero al tercero salen del primer color, del cuarto al sexto del segundo color y el séptimo y octavo del último color.

Ejemplifiquemos el algoritmo:

In [6]:
algo_test = [100, 100]
print pixel_bin(pix, algo_test) 

['00101010', '00110001', '00100001', '11111111']


Es decir, el pixel (100, 100) es de color ['00101010', '00110001', '00100001']. Ahora, el algoritmo dice que <b>"siendo los 3 bits de los primeros de los 2 primeros colores"</b>, por lo tanto necesitamos sacar 3 bits de los 2 primeros colores ('00101010' y '00110001'), pero <b>¿Qué bits deberíamos sacar?</b>.

Si el lector sabe como leer un número binario, entonces también sabe que los términos que menos afectan el número representado son los del extremo derecho. Veamos un ejemplo:

In [7]:
print int('11111111', 2) # Función para pasar de bin a int.
print int('11111110', 2)
print int('01111111', 2)

255
254
127


Modificar el término del extremo derecho sólo resta 1 al total (- 0.4%), mientras que modificar el extremo izquierdo resta <b>128</b> del total (- 50.2%).

Por lo tanto, es lógico que para <i>generar el <b>menor</b> impacto posible</i> en los colores de los pixeles, se deberán modificar los términos del extremo <b>derecho</b> de los bytes de colores.

## Importar Mensaje Oculto

Resulta incómodo y poco práctico escribir el mensaje oculto en la consola de python, por lo que tomaremos un archivo de texto (.txt), lo importaremos y lo ocultaremos en nuestra imágen.

In [8]:
mensaje = open('text.txt', 'r')

El algoritmo dice <i>"Cada pixel oculta un carácter"</i> por lo que nuestro mensaje no puede tener más caracteres que pixeles en nuestra imagen.

In [9]:
def check_length(image, text):
    # Esta función checkea si el texto cae en la imagen.
    # En caso de que caiga, también entrega el porte de la imagen
    # y del texto.
    x, y = image.size
    img_pixels = x * y
    text_chars = 0
    for l in text:
        text_chars += len(l)
    if img_pixels < text_chars:
        return False, []
    return True, [img_pixels, text_chars]

# Test
if check_length(im, mensaje)[0]:
    print 'El mensaje cae en la imagen.'
else:
    print 'El mensaje no cae en la imagen.'

El mensaje cae en la imagen.


Si el mensaje cae en la imagen, es importante saber hasta <i>"en que pixel"</i> puede ser ingresado.

## Claves de Entrada y Salida

No necesariamente el mensaje debe empezar en el primer pixel ni terminar en el último, por lo que definiremos claves de entrada y salida. El programa estará constantemente checkeando si es que aparecen las claves con el fin de empezar a leer o terminar de leer. 

In [10]:
claves = ['hola1234', 'chao5678']

Ahora, es necesario checkear si el mensaje más las claves cae en la imagen, y si es así, hasta en que pixel podemos empezar nuestro mensaje.

In [11]:
def check_position(image, text, claves):
    check = check_length(image, text)
    if check[0]:
        total_length = check[1][1] + len(claves[0]) + len(claves[1])
        
        if total_length > check[1][0]:
            return False, 0
        
        return True, check[1][0] - total_length
    
        
# Cerramos y abrimos para poder volver a explorar.
mensaje.close()
mensaje = open('text.txt', 'r')

# Test
if check_position(im, mensaje, claves)[0]:
    print 'El mensaje más claves cae en la imagen.'
    print 'Último pixel para empezar:', \
            check_position(im, mensaje, claves)[1]


El mensaje más claves cae en la imagen.
Último pixel para empezar: 623984


## Salvar Copia Original

Es importante crear una copia de la imagen original como medida de seguridad y por si se quiere modificar de vuelta la imagen codificada.

In [12]:
from shutil import copyfile # Función copypaste

copyfile('dog_img.png', 'dog_img_modificado.png')

## Modificación de Pixeles

Es el momento de comenzar a modificar los pixeles que abarcarán nuestro mensaje.

In [15]:
def modificar_imagen(img_path, text_path, claves, aqui):
    # aqui: pixel de entrada del mensaje.
    mensaje = open(text_path, 'r') # Abrimos el mensaje.
    im = Image.open(img_path) # Abrimos la imagen
    pix = im.load() # Abrimos los pixeles.
    
    check = check_position(im, mensaje, claves) # Check si cae.
    if check[0]:
        maximo = check[1]
        
        if aqui > maximo:
            print 'El mensaje no cae desde la posición especificada.'
            return 0
        
        # Cerramos y abrimos para poder volver a explorar.
        mensaje.close()
        mensaje = open(text_path, 'r')
        
        # Creamos un string largo con todo el mensaje y las claves.
        msg = claves[0] + ''
        for l in mensaje:
            msg += l
        msg += claves[1]

        # Modificamos
        actual_img, actual_msg = 0, 0 # Posicion en imagen y en mensaje
        # Iteramos en los pixeles hasta llegar a pixel de inicio.
        for i in range(im.size[0]):
            for j in range(im.size[1]):
                actual_img += 1
                # Modificamos hasta que llegamos al final del mensaje.
                if actual_img >= aqui and actual_msg < len(msg):
                    binario = char_int_bin(msg[actual_msg])
                    colores = pixel_bin(pix, [i,j])

                    # Modificamos según algoritmo (a)
                    colores[0] = colores[0][:5] + binario[:3]
                    colores[1] = colores[1][:5] + binario[3:6]
                    colores[2] = colores[2][:6] + binario[6:8]
                    new_color = (int(colores[0], 2),
                                 int(colores[1], 2),
                                 int(colores[2], 2))
                    
                    # Guardamos nuevo pixel
                    pix[i,j] = new_color
                    actual_msg += 1

                elif actual_msg != 0:
                    # Guardamos imagen modificada y salimos.
                    im.save(img_path)
                    mensaje.close()
                    return 1
    else:
        print 'El mensaje no cae en la imagen.'
        return 0



modificar_imagen('dog_img_modificado.png', 'text.txt', claves, 76851)

1

## Decodificación de Mensaje Oculto

Nuestro decodificador debe identificar la clave de entrada y de salida para comenzar a decodificar y terminar de hacerlo.

In [18]:
def traducir(binarios):
    bin_final = binarios[0][-3:] + binarios[1][-3:] + binarios[2][-2:]
    return chr(int(bin_final, 2))

def decodificador(img_path, claves):
    im = Image.open(img_path) # Abrimos la imagen
    pix = im.load() # Abrimos los pixeles
    
    actual_img = 0
    clave_check = ''
    status = 0 
    msg = ''
    for i in range(im.size[0]):
        for j in range(im.size[1]):
            colores = pixel_bin(pix, [i,j])
            clave_check += traducir(colores)
            
            if status == 1:
                msg += traducir(colores)
            
            # Lo mantenemos del largo de la clave
            if len(clave_check) > len(claves[status]):
                clave_check = clave_check[-len(claves[0]):]
                
            if clave_check == claves[status]: # Checkear por clave
                status += 1
                if status == 2:
                    return msg[:-len(claves[1])] # Le sacamos la clave de salida
    return 0

print decodificador('dog_img_modificado.png', claves)

"Me Reclama"
(feat. Lui-G 21 Plus)

[Ozuna:]
Le estoy mirando desde su entrada 
Es que esa shorty como que me gusto 
Disimulando pero cambiamos miradas 
Y como si nada se me pego 

Le pregunte su nombre 
Lo mismo me preguntaba 
Ella quería que yo fuera su hombre 
Y yo quería que ella fuera mi dama 

Yo soy el que le gusta 
Su cuerpo me reclama 
Cuando se siente sola 
Yo soy el hombre que ella llama [x2] 

[Luigi 21 Plus:]
El hombre que ella siempre llama 
¡Si ese soy yo! 
El que el chupa bien rico, ese bowyou 
El que ella quiere, el que más desea 
Con el único que chinga, a la hora que sea 

Es que ella los prefiere, boquisucios 
Quiere que le meta duro con el prepucio 
Se puso en cuatro patas, quiere que lo entre 
Por donde le sale ca**, abre esas patas 

Yo soy su hombre, ella es mi lady 
Yo soy su puto, ella es mi baby 
Cuando está sola es a mí el que llama 
Soy su gato favorito en la cama 

Yo soy el que le gusta 
Su cuerpo me reclama 
Cuando se siente sola 
Yo soy el hombre que el

# Compración de Imágenes

<figure>
    <img src="dog_img.png", width=600, alt="dog_img">
    <figcaption>Imagen Original</figcaption>
</figure>

<figure>
    <img src="dog_img_modificado.png", width=600, alt="dog_img_mod">
    <figcaption>Imagen Modificada</figcaption>
</figure>